# Professional Job Application Toolkit

A streamlined toolkit for email sending and cover letter generation.

## Features
- Email sending with your existing email template
- Cover letter generation with your existing template  
- PDF export functionality
- Simple variable input system

## 1. Import Libraries and Configuration

In [186]:
# Import required libraries
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
from docx.oxml.shared import OxmlElement, qn
from dotenv import load_dotenv

# For PDF conversion
try:
    from docx2pdf import convert
    PDF_CONVERSION_AVAILABLE = True
    print("✅ PDF conversion library available")
except ImportError:
    PDF_CONVERSION_AVAILABLE = False
    print("⚠️ docx2pdf not available. Install with: pip install docx2pdf")

# Load environment variables from .env file
load_dotenv()

# Gmail Configuration
GMAIL_EMAIL = os.getenv('GMAIL_EMAIL')
GMAIL_PASSWORD = os.getenv('GMAIL_PASSWORD')
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# Contact Information
FULL_NAME = "Felix Lau Pangestu"
PHONE_NUMBER = os.getenv('PHONE_NUMBER')
LINKEDIN_URL = "linkedin.com/in/jfelixpangestu/"

✅ PDF conversion library available


## 2. Template Functions (From Your LetterMaker.py)

In [187]:
def coverLetterBody(company_name, role, industry, company_specific_interest=None, referral_name=None):
    """
    Generate interest-focused cover letter emphasizing company fit and passion
    
    Parameters:
    - company_name: Target company name  
    - role: Specific position title
    - industry: Industry sector
    - company_specific_interest: What specifically attracts you to this company (optional)
    - referral_name: Name of person who referred you (optional)
    """
    
    # Opening with genuine interest
    if referral_name:
        opening = f"I am writing to apply for the {role} position at {company_name}. {referral_name} spoke highly of the team and culture, which reinforced my strong interest in joining your organization."
    else:
        opening = f"I am writing to express my enthusiasm for the {role} position at {company_name}."
    
    # Company-specific interest or general industry passion
    if company_specific_interest:
        interest_statement = company_specific_interest
    else:
        interest_statements = {
            "investment banking": "Your firm's reputation for excellence and commitment to helping clients navigate complex financial challenges",
            "technology": "Your innovative approach to solving complex problems and building products that impact millions of users",
            "fintech": "Your mission to democratize financial services through technology and create more accessible financial solutions",
            "consulting": "Your collaborative approach to solving business challenges and commitment to client success",
            "asset management": "Your focus on delivering superior investment outcomes and fiduciary responsibility to clients"
        }
        interest_statement = interest_statements.get(industry, f"Your leadership position and innovative approach in the {industry} industry")
    
    # Skills paragraph - high level alignment
    skills_para = f"""As a final-year Business Computing & Data Analytics student at Hong Kong Baptist University, I have developed expertise in full-stack web development, database management, and machine learning through internships at organizations like Cathay Pacific and Wizpresso. My experience building scalable applications, optimizing system performance, and working with large datasets has prepared me well for the technical challenges at {company_name}. Beyond the technical skills, I've had the opportunity to collaborate with cross-functional teams, lead projects, and communicate complex technical concepts to stakeholders - experiences that align well with the collaborative nature of your {role} position."""
    
    # Passion and culture fit paragraph
    culture_para = f"""What excites me most about {company_name} is the opportunity to work alongside talented professionals who share a passion for {industry}. I'm particularly drawn to environments where I can continue learning, take on meaningful challenges, and contribute to projects that create real impact. My entrepreneurial experience developing technology solutions for social good, combined with my leadership roles as Student Ambassador and AWS Student Ambassador, reflects my drive to make a positive difference - values I see reflected in {company_name}'s mission and culture."""
    
    # Closing with enthusiasm
    closing = f"""I would be thrilled to contribute to {company_name}'s continued success and would welcome the opportunity to discuss how my passion for {industry} and technical background can add value to your team. Thank you for considering my application."""
    
    # Combine all paragraphs
    output = f"""{opening} {interest_statement} particularly resonates with me, and I am excited about the opportunity to contribute to your team's success.

{skills_para}

{culture_para}

{closing}"""
    
    return output


#### Separation

def emailLetter(company_name, role):
    """Generate email content"""
    output = f"""Dear {company_name} Team,

I am currently studying at Hong Kong Baptist University, majoring in Business Computing & Data Analytics, from which I will be graduating in May 2026. It is with great interest that I am applying for the {role} position at {company_name}.

I am confident that my technical qualifications and work experiences developing solutions for clients and end users make me an excellent candidate for this internship. I would appreciate the opportunity to discuss how I can help support {company_name}'s needs.

Thank you for your consideration.
Attached you will find my CV and recommendation letters.

Yours sincerely,
Felix Lau Pangestu
"""
    return output

print("Template functions loaded successfully!")

# Usage examples:
print("\nUsage Examples:")
print("# Standard application:")
print('coverLetterBody("Goldman Sachs", "Technology Analyst", "investment banking", company_specific_interest="Your commitment to digital innovation in capital markets and recent investments in blockchain technology")')
print("\n# With referral:")
print('coverLetterBody("Meta", "Software Engineer", "technology", company_specific_interest="Your focus on building the metaverse and connecting people through immersive experiences")')
print("\n# With referral:")
print('coverLetterBody("JPMorgan Chase", "Software Engineer", "fintech", referral_name="Sarah Chen")')

Template functions loaded successfully!

Usage Examples:
# Standard application:
coverLetterBody("Goldman Sachs", "Technology Analyst", "investment banking", company_specific_interest="Your commitment to digital innovation in capital markets and recent investments in blockchain technology")

# With referral:
coverLetterBody("Meta", "Software Engineer", "technology", company_specific_interest="Your focus on building the metaverse and connecting people through immersive experiences")

# With referral:
coverLetterBody("JPMorgan Chase", "Software Engineer", "fintech", referral_name="Sarah Chen")


In [188]:
# Test the new enhanced coverLetterBody function with different scenarios

print("🔍 TESTING NEW COVER LETTER FUNCTION")
print("=" * 60)

# Example 1: Standard application
print("\n1️⃣ STANDARD APPLICATION (Goldman Sachs):")
print("-" * 40)
example1 = coverLetterBody("Goldman Sachs", "Technology Analyst", "investment banking", 
                          source="LinkedIn", company_specific="recent expansion into digital assets")
print(example1[:300] + "...")

# Example 2: With referral
print("\n2️⃣ WITH REFERRAL (Meta):")
print("-" * 40)
example2 = coverLetterBody("Meta", "Software Engineer", "technology", 
                          referral_name="John Smith", referral_department="Engineering",
                          tech_stack="React, Python, distributed systems")
print(example2[:300] + "...")

# Example 3: With tech stack (JPMorgan)
print("\n3️⃣ WITH TECH STACK (JPMorgan Chase):")
print("-" * 40)
example3 = coverLetterBody("JPMorgan Chase", "Software Engineer", "fintech",
                          source="career fair", 
                          company_specific="commitment to modernizing financial infrastructure",
                          tech_stack="Java, Spring Boot, AWS")
print(example3[:300] + "...")

print("\n✅ All examples generated successfully!")
print("📝 Full letters saved to variables: example1, example2, example3")

🔍 TESTING NEW COVER LETTER FUNCTION

1️⃣ STANDARD APPLICATION (Goldman Sachs):
----------------------------------------


TypeError: coverLetterBody() got an unexpected keyword argument 'source'

## 3. Input Variables

In [ ]:
# ===== INPUT VARIABLES - MODIFY THESE FOR EACH APPLICATION =====

# Company and position details
company_name = 'Cathay'
position = 'Graduate Trainee'
industry = 'aviation'
recipient_email = 'chokinghazarddd@gmail.com'

# COVER LETTER PARAMETERS
company_specific_interest = "commitment to sustainable aviation and innovation in air cargo operations"  # What specifically attracts you to this company
referral_name = None  # e.g., "John Smith" (if referred by someone)

# Email details
email_subject = f'Application for {position} Position'  # Email subject

# Attachment paths
attachment_paths = ['Felix Lau Pangestu_CV.pdf',f'cover_letters\cover_letter_{company_name}.pdf', 'Transcript_HKBU.pdf', 'Recommendation_Letters.pdf']  # List of file paths to attach

print("Variables configured:")
print(f"Company: {company_name}")
print(f"Position: {position}")
print(f"Industry: {industry}")
print(f"Company specific interest: {company_specific_interest}")
print(f"Referral name: {referral_name}")
print(f"Recipient: {recipient_email}")
print(f"Subject: {email_subject}")

print("\nTIP: Customize for each application:")
print("   - company_specific_interest: What specifically attracts you to this company")
print("   - referral_name: If you have a referral")

Variables configured:
Company: Cathay
Position: Graduate Trainee
Industry: aviation
Company specific interest: commitment to sustainable aviation and innovation in air cargo operations
Referral name: None
Recipient: chokinghazarddd@gmail.com
Subject: Application for Graduate Trainee Position

TIP: Customize for each application:
   - company_specific_interest: What specifically attracts you to this company
   - referral_name: If you have a referral


## 4. Email Sending Function

In [ ]:
def send_email(to_email, subject, body, attachment_path=None):
    """Send email using Gmail SMTP"""
    try:
        # Create message
        msg = MIMEMultipart()
        msg['From'] = GMAIL_EMAIL
        msg['To'] = to_email
        msg['Subject'] = subject
        
        # Add body
        msg.attach(MIMEText(body, 'plain'))
        
        # Initialize attachment path as list if not already
        if attachment_path and not isinstance(attachment_path, list):
            attachment_path = [attachment_path]
        elif not attachment_path:
            attachment_path = []

        # Add multiple attachments if provided
        for single_attachment in attachment_path:
            if os.path.exists(single_attachment):
                with open(single_attachment, "rb") as attachment:
                    part = MIMEBase('application', 'octet-stream')
                    part.set_payload(attachment.read())
                
                encoders.encode_base64(part)
                part.add_header(
                    'Content-Disposition',
                    f'attachment; filename= {os.path.basename(single_attachment)}'
                )
                msg.attach(part)
                print(f"📎 Attachment added: {os.path.basename(single_attachment)}")
            else:
                print(f"⚠️ Attachment not found: {single_attachment}")
        
        # Send email
        context = ssl.create_default_context()
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls(context=context)
            server.login(GMAIL_EMAIL, GMAIL_PASSWORD)
            server.sendmail(GMAIL_EMAIL, to_email, msg.as_string())
        
        print(f"✅ Email sent successfully to {to_email}")
        return True
        
    except Exception as e:
        print(f"❌ Failed to send email: {str(e)}")
        return False

## 5. PDF Export Function

In [ ]:
def export_cover_letter_to_word_and_pdf(company_name, role, industry, word_filename=None, pdf_filename=None,
                                        company_specific_interest=None, referral_name=None):
    """Export cover letter to Word document and PDF with interest-focused formatting"""
    try:
        # Create cover_letters directory if it doesn't exist (only for PDF)
        cover_letters_dir = "cover_letters"
        if not os.path.exists(cover_letters_dir):
            os.makedirs(cover_letters_dir)
            print(f"Created directory: {cover_letters_dir}")
        
        # Set default filenames if not provided
        if word_filename is None:
            word_filename = "Cover_Letter.docx"  # Save in main directory
        if pdf_filename is None:
            pdf_filename = f"cover_letter_{company_name}.pdf"
            
        # Create full paths
        word_path = word_filename  # Save Word doc in main directory
        pdf_path = os.path.join(cover_letters_dir, pdf_filename)  # Save PDF in cover_letters folder
        
        # Create a new document
        doc = Document()
        
        # Set up document margins (1 inch on all sides)
        sections = doc.sections
        for section in sections:
            section.top_margin = Inches(0.5)
            section.bottom_margin = Inches(0.5)
            section.left_margin = Inches(1)
            section.right_margin = Inches(1)
        
        # Header with centered name and contact info combined
        header = doc.add_paragraph()
        header.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Add name
        name_run = header.add_run(FULL_NAME)
        name_run.font.name = 'Times New Roman'
        name_run.font.size = Pt(16)
        name_run.bold = True
        
        # Add line break and contact info
        header.add_run('\n')
        contact_run = header.add_run(f"{PHONE_NUMBER} | {GMAIL_EMAIL} | {LINKEDIN_URL}")
        contact_run.font.name = 'Times New Roman'
        contact_run.font.size = Pt(11)
    
        # Date
        current_date = datetime.now().strftime("%B %d, %Y")
        date_para = doc.add_paragraph(current_date)
        date_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        # Recipient information
        recipient_para = doc.add_paragraph()
        recipient_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        recipient_para.add_run(f"{company_name}\n").bold = True

        # Salutation
        salutation_para = doc.add_paragraph("To Whom it May Concern,")
        salutation_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        # Get the body content
        body_content = coverLetterBody(company_name, role, industry, 
                                     company_specific_interest=company_specific_interest,
                                     referral_name=referral_name)
        
        # Split body into paragraphs and add them
        paragraphs = body_content.strip().split('\n\n')
        
        for i, paragraph in enumerate(paragraphs):
            if paragraph.strip():
                # Check if it's the closing paragraph
                if paragraph.strip().startswith("Thank you for considering") or paragraph.strip().startswith("I would be thrilled"):
                    # Add spacing before closing
                    closing_para = doc.add_paragraph(paragraph.strip())
                    closing_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
                else:
                    # Regular body paragraph
                    body_para = doc.add_paragraph(paragraph.strip())
                    body_para.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        

        # Add closing salutation
        closing_salutation = doc.add_paragraph("Sincerely,")
        closing_salutation.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        # Add signature space
        doc.add_paragraph()
        doc.add_paragraph()
        
        # Add signature line
        signature_para = doc.add_paragraph(FULL_NAME)
        signature_para.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        # Save the Word document in main directory
        doc.save(word_path)
        print(f"Word document created: {word_path}")
        
        # Convert to PDF and save in cover_letters folder if library is available
        if PDF_CONVERSION_AVAILABLE:
            try:
                convert(word_path, pdf_path)
                print(f"PDF document created: {pdf_path}")
                return {'word_path': os.path.abspath(word_path), 'pdf_path': os.path.abspath(pdf_path)}
            except Exception as e:
                print(f"PDF conversion failed: {str(e)}")
                return {'word_path': os.path.abspath(word_path), 'pdf_path': None}
        else:
            print("PDF conversion skipped - docx2pdf not available")
            print("   Install with: pip install docx2pdf")
            return {'word_path': os.path.abspath(word_path), 'pdf_path': None}
        
    except Exception as e:
        print(f"Failed to create documents: {str(e)}")
        return {'word_path': None, 'pdf_path': None}

print("Word document and PDF export functions ready!")

Word document and PDF export functions ready!


In [ ]:
# Generate cover letter and export to Word and PDF using new parameters
export_result = export_cover_letter_to_word_and_pdf(company_name, position, industry,
                                                   company_specific_interest=company_specific_interest,
                                                   referral_name=referral_name)

# Store paths for later use
word_path = export_result['word_path']
pdf_path = export_result['pdf_path']

print("\nFILES CREATED:")
if word_path:
    print(f"Word document: {word_path}")
if pdf_path:
    print(f"PDF document: {pdf_path}")
else:
    print("PDF not created - install docx2pdf library")

Word document created: Cover_Letter.docx


  0%|          | 0/1 [00:00<?, ?it/s]

PDF document created: cover_letters\cover_letter_Cathay.pdf

FILES CREATED:
Word document: c:\Users\j_fel\Desktop\jobApplications\Cover_Letter.docx
PDF document: c:\Users\j_fel\Desktop\jobApplications\cover_letters\cover_letter_Cathay.pdf


In [ ]:
# Test the new interest-focused function with different examples

print("TESTING NEW INTEREST-FOCUSED COVER LETTER FUNCTION")
print("=" * 60)

# Example 1: Goldman Sachs with specific interest
print("\n1. GOLDMAN SACHS (Investment Banking):")
print("-" * 40)
example1 = coverLetterBody("Goldman Sachs", "Technology Analyst", "investment banking", 
                          company_specific_interest="Your commitment to digital innovation in capital markets and recent investments in blockchain technology")
print(example1[:300] + "...")

# Example 2: Meta with specific interest
print("\n2. META (Technology):")
print("-" * 40)
example2 = coverLetterBody("Meta", "Software Engineer", "technology", 
                          company_specific_interest="Your focus on building the metaverse and connecting people through immersive experiences")
print(example2[:300] + "...")

# Example 3: JPMorgan with referral
print("\n3. JPMORGAN CHASE (With Referral):")
print("-" * 40)
example3 = coverLetterBody("JPMorgan Chase", "Software Engineer", "fintech", referral_name="Sarah Chen")
print(example3[:300] + "...")

# Example 4: Using default industry passion (no specific interest)
print("\n4. DELOITTE (Default Industry Passion):")
print("-" * 40)
example4 = coverLetterBody("Deloitte", "Technology Consultant", "consulting")
print(example4[:300] + "...")

print("\n" + "=" * 60)
print("New interest-focused function working perfectly!")
print("Key improvements:")
print("- Simplified parameters (only 2 optional ones)")
print("- Emphasis on genuine interest and company fit")
print("- Built-in industry-specific passion statements")
print("- Focus on culture and learning opportunities")

TESTING NEW INTEREST-FOCUSED COVER LETTER FUNCTION

1. GOLDMAN SACHS (Investment Banking):
----------------------------------------
I am writing to express my enthusiasm for the Technology Analyst position at Goldman Sachs. Your commitment to digital innovation in capital markets and recent investments in blockchain technology particularly resonates with me, and I am excited about the opportunity to contribute to your team's suc...

2. META (Technology):
----------------------------------------
I am writing to express my enthusiasm for the Software Engineer position at Meta. Your focus on building the metaverse and connecting people through immersive experiences particularly resonates with me, and I am excited about the opportunity to contribute to your team's success.

As a final-year Bus...

3. JPMORGAN CHASE (With Referral):
----------------------------------------
I am writing to apply for the Software Engineer position at JPMorgan Chase. Sarah Chen spoke highly of the team and cul

In [ ]:
# 🎯 QUICK TEST: Generate a sample cover letter with different parameters
print("🎯 QUICK TEST: Different Company Example")
print("=" * 60)

# Test with a tech company that requires specific tech stack
test_letter = coverLetterBody(
    company_name="Meta", 
    role="Software Engineer Intern", 
    industry="technology",
    source="LinkedIn",
    company_specific="recent innovations in AI and the metaverse",
    tech_stack="React, Python, Machine Learning",
    hook_type="achievement"
)

print("📝 SAMPLE OUTPUT (Meta - Software Engineer):")
print("-" * 50)
print(test_letter[:800] + "...")

print("\n" + "=" * 60)
print("✅ Enhanced cover letter function working perfectly!")
print("💡 You can now customize letters for any company with:")
print("   • Different sources (LinkedIn, career fair, referral)")
print("   • Company-specific mentions")
print("   • Tech stack requirements")
print("   • Referral information")
print("   • Various opening hooks")

🎯 QUICK TEST: Different Company Example
📝 SAMPLE OUTPUT (Meta - Software Engineer):
--------------------------------------------------
I am writing to apply for the Software Engineer Intern position at Meta. I learned about this opportunity through LinkedIn and am drawn to Meta because of your recent innovations in AI and the metaverse. As a final-year Business Computing & Data Analytics student at Hong Kong Baptist University with demonstrated expertise in full-stack development and performance optimization, I am excited to contribute to your team's objectives.

During my recent internship at Cathay Pacific, I built a full-stack cargo compliance MVP using React/Flask that identified HKD 10M+ in risk mitigation opportunities for 40K+ monthly packets. I implemented database indexing strategies and query optimization techniques that reduced data processing time from 2+ hours to 6 minutes, and developed an automated rosterin...

✅ Enhanced cover letter function working perfectly!
💡 You ca

## NEW INTEREST-FOCUSED COVER LETTER FUNCTION

### Updated `coverLetterBody()` Function

The cover letter function now emphasizes genuine interest and company fit with a simplified, clean approach.

#### **New Simplified Parameters:**
- **`company_specific_interest`**: What specifically attracts you to this company (optional)
- **`referral_name`**: Name of person who referred you (optional)

#### **Key Features:**
- **Interest-focused opening** emphasizing enthusiasm for the role
- **Built-in industry passion statements** for common sectors (investment banking, technology, fintech, consulting, asset management)
- **Company fit emphasis** highlighting alignment with company culture and values
- **Skills integration** showing relevant technical and soft skills
- **Culture paragraph** demonstrating passion for learning and impact
- **Clean, professional tone** without excessive metrics or buzzwords

#### **Usage Examples:**
```python
# Standard application with company-specific interest
coverLetterBody("Goldman Sachs", "Technology Analyst", "investment banking", 
               company_specific_interest="Your commitment to digital innovation in capital markets and recent investments in blockchain technology")

# With custom company interest
coverLetterBody("Meta", "Software Engineer", "technology", 
               company_specific_interest="Your focus on building the metaverse and connecting people through immersive experiences")

# With referral (uses default industry passion)
coverLetterBody("JPMorgan Chase", "Software Engineer", "fintech", referral_name="Sarah Chen")

# Using built-in industry statements
coverLetterBody("Deloitte", "Technology Consultant", "consulting")
```

#### **Built-in Industry Statements:**
- **Investment Banking**: "Your firm's reputation for excellence and commitment to helping clients navigate complex financial challenges"
- **Technology**: "Your innovative approach to solving complex problems and building products that impact millions of users"  
- **Fintech**: "Your mission to democratize financial services through technology and create more accessible financial solutions"
- **Consulting**: "Your collaborative approach to solving business challenges and commitment to client success"
- **Asset Management**: "Your focus on delivering superior investment outcomes and fiduciary responsibility to clients"

## 6. Generate Content

In [ ]:
# Generate email content
email_content = emailLetter(company_name, position)

print("\nEMAIL CONTENT:")
print("=" * 50)
print(email_content[:500] + "..." if len(email_content) > 500 else email_content)

# Generate cover letter body content for preview using new simplified parameters
cover_letter_body = coverLetterBody(company_name, position, industry, 
                                   company_specific_interest=company_specific_interest,
                                   referral_name=referral_name)

print("\nCOVER LETTER BODY CONTENT:")
print("=" * 50)
print(cover_letter_body[:500] + "..." if len(cover_letter_body) > 500 else cover_letter_body)


EMAIL CONTENT:
Dear Cathay Team,

I am currently studying at Hong Kong Baptist University, majoring in Business Computing & Data Analytics, from which I will be graduating in May 2026. It is with great interest that I am applying for the Graduate Trainee position at Cathay.

I am confident that my technical qualifications and work experiences developing solutions for clients and end users make me an excellent candidate for this internship. I would appreciate the opportunity to discuss how I can help support Ca...

COVER LETTER BODY CONTENT:
I am writing to express my enthusiasm for the Graduate Trainee position at Cathay. commitment to sustainable aviation and innovation in air cargo operations particularly resonates with me, and I am excited about the opportunity to contribute to your team's success.

As a final-year Business Computing & Data Analytics student at Hong Kong Baptist University, I have developed expertise in full-stack web development, database management, and machine l

## 8. Send Email (Uncomment to Execute)

In [ ]:
# SEND EMAIL - Uncomment the line below to actually send the email
send_email(recipient_email, email_subject, email_content, attachment_paths)

print("⚠️  Email sending is commented out for safety.")
print("⚠️  Uncomment the send_email line above to actually send the email.")
print("\n📋 SUMMARY:")
print(f"✅ Email content generated using your template")
print(f"✅ Cover letter content generated using your template")
print(f"✅ Word document exported: {word_path if word_path else 'Failed'}")
print(f"✅ PDF document exported: {pdf_path if pdf_path else 'Failed - install docx2pdf'}")
print(f"🎯 Ready to send to: {recipient_email}")
print(f"📧 Subject: {email_subject}")
print(f"📎 Attachments: {', '.join([os.path.basename(path) for path in attachment_paths]) if attachment_paths else 'None'}")

print("\n📄 TO USE THIS TOOLKIT:")
print("1. Modify the variables in Section 3")
print("2. Run all cells")
print("3. Uncomment the send_email line above to send")
print("4. Install docx2pdf for PDF conversion: pip install docx2pdf")

print(f"\n✅ All files ready in: {os.getcwd()}")
print(f"📁 Cover letters saved in: {os.path.join(os.getcwd(), 'cover_letters')}")

📎 Attachment added: Felix Lau Pangestu_CV.pdf
📎 Attachment added: cover_letter_Cathay.pdf
📎 Attachment added: Transcript_HKBU.pdf
📎 Attachment added: Recommendation_Letters.pdf
✅ Email sent successfully to chokinghazarddd@gmail.com
⚠️  Email sending is commented out for safety.
⚠️  Uncomment the send_email line above to actually send the email.

📋 SUMMARY:
✅ Email content generated using your template
✅ Cover letter content generated using your template
✅ Word document exported: c:\Users\j_fel\Desktop\jobApplications\Cover_Letter.docx
✅ PDF document exported: c:\Users\j_fel\Desktop\jobApplications\cover_letters\cover_letter_Cathay.pdf
🎯 Ready to send to: chokinghazarddd@gmail.com
📧 Subject: Application for Graduate Trainee Position
📎 Attachments: Felix Lau Pangestu_CV.pdf, cover_letter_Cathay.pdf, Transcript_HKBU.pdf, Recommendation_Letters.pdf

📄 TO USE THIS TOOLKIT:
1. Modify the variables in Section 3
2. Run all cells
3. Uncomment the send_email line above to send
4. Install docx2pd